In [ ]:
import os
os.chdir('../')  # Moving up one directory to the root
import sys
import tensorflow as tf
import numpy as np
import random
import json
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from models.sentiment_model import EnhancedDistilBertForSentiment
from config.model_config import ModelConfig
from main import SentimentAnalyzer
from utils.analysis import SentimentAnalysisVisualizer

In [ ]:
# Add project root to Python path
project_root = os.path.dirname(os.getcwd())
sys.path.append(project_root)

In [ ]:
# Initialize analyzer and load model
analyzer = SentimentAnalyzer()
model, history = analyzer.load_saved_model(epoch=5)

In [ ]:
# Visualize training history
visualizer = SentimentAnalysisVisualizer()
visualizer.visualize_training_history(history)

In [ ]:
# Load and prepare test data
analyzer.process_data()

# Get predictions for all test texts
test_predictions = []
for text in analyzer.test_texts:
    prediction = analyzer.predict(text)  # This already handles tokenization internally
    test_predictions.append(prediction)


# Evaluate results
y_true = np.argmax(analyzer.test_labels['sentiment'], axis=1)
y_pred = [np.argmax([p['sentiment']['negative'], p['sentiment']['neutral'], p['sentiment']['positive']]) 
          for p in test_predictions]

# Calculate metrics
print("\nClassification Report:")
target_names = ['Negative', 'Neutral', 'Positive']
print(classification_report(y_true, y_pred, target_names=target_names))

# Show some example predictions
print("\nSample Predictions from Test Set:")
sample_indices = random.sample(range(len(analyzer.test_texts)), 5)
for idx in sample_indices:
    text = analyzer.test_texts[idx]
    true_sentiment = np.argmax(analyzer.test_labels['sentiment'][idx])
    print(f"\nText: {text}")
    print(f"True Sentiment: {target_names[true_sentiment]}")
    print("Predictions:", json.dumps(test_predictions[idx], indent=2))